# Part 2: Hardware Demo
## Outline
1. RPC server and client  
2. Signal generation and oscilloscope measurement  
    2.1 Pulse generation  
    2.2 Gate generation  
    2.3 Circuit generation  
3. Loopback test  
4. Readout emulator and GMM fitting  
5. Fast feedback - active reset  

In [ ]:
import qubic.toolchain as tc
import qubic.rpc_client as rc
import qubitconfig.qchip as qc
from distproc.hwconfig import FPGAConfig, load_channel_configs
import chipcalibration.config as cfg
import numpy as np
import matplotlib.pyplot as plt
#import qubic.run
import qubic.state_disc as sd
from chipcalibration import vna as vn
import qubic.job_manager as jm

## 2. Signal generation and oscilloscope measurement
### 2.1 Pulse generation

### Load configs and define circuit
Using the chipcalibration repository, load all three configs:  
a. FPGA config: this has timing information for the scheduler  
b. Channel configs: firmware channel mapping + configuration, see [Understanding Channel Configuration](https://gitlab.com/LBL-QubiC/software/-/wikis/Understanding-Channel-Configuration) for details  
c. QChip object: contains calibrated gates + readout  

In [ ]:
fpga_config = FPGAConfig(**{'fpga_clk_period': 2.e-9, 'alu_instr_clks': 5, 
                            'jump_cond_clks': 5, 'jump_fproc_clks': 5, 
                            'pulse_regwrite_clks': 3, 'pulse_load_clks': 4})
channel_configs = load_channel_configs('channel_config.json')
qchip = qc.QChip('qubitcfg.json')
#qchip.cfg_dict

Define a circuit at the pulse-level. For details on the QubiC circuit format and supported operations, see [compiler.py](https://gitlab.com/LBL-QubiC/distributed_processor/-/blob/master/python/distproc/compiler.py).

In [ ]:
circuit_1 = [
    
    # play a pi pulse on Q3
    {'name': 'pulse', 'phase': 0, 'freq': 400e6, 'amp': 0.99, 'twidth': 64e-9,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q3.qdrv'},
    
    # play a pi/2 pulse on Q6
    {'name': 'pulse', 'phase': 0, 'freq': 5.7e9, 'amp': 0.50, 'twidth': 32e-9,
    'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
    'dest': 'Q6.qdrv'}

]

### Compile and assemble

Compile the program. The output of the compile stage is a distributed processor assembly program, which consists of initialization / termination statements, as well as a list of scheduled pulses for each core.

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_1, fpga_config, qchip)
#compiled_prog.program

Run the assembler to convert the above program into machine code that we can load onto the FPGA.

In [ ]:
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)

### Connect to server and run circuit

Now that we've defined our circuit and compiled it to machine code, we can submit it to the ZCU216 and run it.

Instantiate the runner client.

In [ ]:
#runner = qubic.run.CircuitRunner(commit='f24f1615')
runner = rc.CircuitRunnerClient(ip='localhost', port=9090)

Submit the circuit to the server, and collect 1 shot.

In [ ]:
acq_data = runner.load_and_run_acq(raw_asm, n_total_shots=1, acq_chans=['0','1'], trig_delay=0e-9)

Observe the pulses on the oscilloscope or through the acq buffer.

In [ ]:
time_step=0.5e-9
plt.plot(np.arange(0,acq_data['1'].shape[1]*time_step,time_step)[400:700],np.average(acq_data['1'], axis=0)[400:700])

### 2.2 Gate generation
Define a circuit with calibrated gates / parameters.

In [ ]:
circuit_2 = [
    
    # play two consecutive pi/2 pulses on Q3
    {'name': 'X90', 'qubit': 'Q3'},
    {'name': 'X90', 'qubit': 'Q3'},
    
    # schedule barrier
    {'name': 'barrier', 'qubit': ['Q3','Q6']},
    
    # play two pi/2 pulses on Q3 and Q6 simultaneously
    {'name': 'X90', 'qubit': 'Q3'},
    {'name': 'X90', 'qubit': 'Q6'},   

    # schedule barrier
    {'name': 'barrier', 'qubit': ['Q3','Q6']},
    
    # play a CNOT gate on Q3 and Q6
    {'name': 'CNOT', 'qubit': ['Q3','Q6']}
    
]

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_2, fpga_config, qchip)
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)

In [ ]:
acq_data = runner.load_and_run_acq(raw_asm, n_total_shots=1, acq_chans=['0','1'], trig_delay=0e-9)
time_step=0.5e-9
plt.plot(np.arange(0,acq_data['1'].shape[1]*time_step,time_step)[400:1400],np.average(acq_data['1'], axis=0)[400:1400])

### 2.3 Circuit generation
Define a quantum circuit.

In [ ]:
circuit_3 = [
    
    # allow qubit to decay to 0 state between shots
    {'name': 'delay', 't': 500.e-9}, 
    
    # set Q3 state to |1>
    {'name': 'X90', 'qubit': 'Q3'},
    {'name': 'X90', 'qubit': 'Q3'},
    
    # set Q6 to a state on the equator 
    {'name': 'X90', 'qubit': 'Q6'},
    
    # schedule barrier ensures that both readouts start after the pulses
    {'name': 'barrier', 'qubit': ['Q3', 'Q6']},
    
    # play readout gates for measurement
    {'name': 'read', 'qubit': 'Q3'},
    {'name': 'read', 'qubit': 'Q6'}

]

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_3, fpga_config, qchip)
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)

In [ ]:
acq_data = runner.load_and_run_acq(raw_asm, n_total_shots=1, acq_chans=['0','1'], trig_delay=0e-9)
time_step=0.5e-9
plt.plot(np.arange(0,acq_data['1'].shape[1]*time_step,time_step)[1400:1700],np.average(acq_data['1'], axis=0)[1400:1700])

## 3. Loopback test
Sweep readout frequency like a VNA.

In [ ]:
fstart=1.0e9
fstop=7.0e9
vna = vn.Vna(0.99, np.linspace(fstart, fstop, 100), 10)
jobman = jm.JobManager(fpga_config, channel_configs, runner, qchip)
vna.run_and_report(jobman)

In [ ]:
plt.plot(vna.freqs, vna.results['amp'])
plt.show()
plt.plot(vna.freqs, vna.results['phase'])
plt.show()

## 4. Readout emulator and GMM fitting
Create a qubit readout emulator.

In [ ]:
fread=2.7568e9
circuit_4 = [
    {'name': 'read', 'qubit': 'Q3', 
     'modi':{(0, 'amp'): 0.99, (0, 'freq'): fread, (1, 'freq'): fread, (1, 'phase'): 0}}
]

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_4, fpga_config, qchip)
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)
s11 = runner.run_circuit_batch([raw_asm], 2000, delay_per_shot=0)

A dictionary of downconverted + integrated complex (IQ) values is returned for each loaded channel. Here, we're using Q3, so we get back data for channel '3'.

In [ ]:
plt.figure()
ax1=plt.subplot(111)
ax1.set_aspect('equal')
plt.plot(s11['3'].real[0], s11['3'].imag[0], '.')
lim=max(1.1*max(max(abs(s11['3'].real[0])),max(abs(s11['3'].imag[0]))),0.1)
ax1.set_xlim([-lim,lim])
ax1.set_ylim([-lim,lim])
plt.grid()

Fit the two blobs with GMM

In [ ]:
gmm_manager = sd.GMMManager(chanmap_or_chan_cfgs=channel_configs)
gmm_manager.fit(s11)
gmm_manager.gmm_dict['Q3'].gmmfit.means_

In [ ]:
angle = gmm_manager.get_threshold_angle('Q3')
circuit_5 = [
    {'name': 'read', 'qubit': 'Q3',
     'modi':{(0, 'amp'): 0.99, (0, 'freq'): fread, (1, 'freq'): fread, (1, 'phase'): np.pi/2 - angle}}
]

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_5, fpga_config, qchip)
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)
s11 = runner.run_circuit_batch([raw_asm], 2000, delay_per_shot=0)

In [ ]:
gmm_manager = sd.GMMManager(chanmap_or_chan_cfgs=channel_configs)
gmm_manager.fit(s11)
plt.figure()
ax1=plt.subplot(111)
ax1.set_aspect('equal')
plt.plot(s11['3'].real[0], s11['3'].imag[0], '.')
lim=max(1.1*max(max(abs(s11['3'].real[0])),max(abs(s11['3'].imag[0]))),0.1)
ax1.set_xlim([-lim,lim])
ax1.set_ylim([-lim,lim])
plt.grid()

## 5. Fast feedback - active reset

In [ ]:
cond_lhs=1 if gmm_manager.gmm_dict['Q3'].gmmfit.means_[0][0]>0 else 0
circuit_6 = [
    {'name': 'X90', 'qubit': 'Q3'},
    {'name': 'read', 'qubit': 'Q3', 
     'modi':{(0, 'amp'): 0.99, (0, 'freq'): fread, (1, 'freq'): fread, (1, 'phase'): np.pi/2 - angle}},
    {'name': 'branch_fproc', 'alu_cond': 'eq', 'cond_lhs': cond_lhs, 'func_id': 3, 'scope': 'Q3',
                'true': [{'name': 'delay', 't': 200.e-9, 'qubit': 'Q3'},
                             {'name': 'X90', 'qubit': 'Q3', 'modi':{(0, 'amp'): 0.99, (0, 'freq'): 900e6}}, 
                             {'name': 'X90', 'qubit': 'Q3', 'modi':{(0, 'amp'): 0.99, (0, 'freq'): 900e6}}], 
                'false': []},
]

In [ ]:
compiled_prog = tc.run_compile_stage(circuit_6, fpga_config, qchip)
raw_asm = tc.run_assemble_stage(compiled_prog, channel_configs)
s11 = runner.run_circuit_batch([raw_asm], 1, delay_per_shot=0)

In [ ]:
print(s11)
gmm_manager.predict(s11)